## A * 

- minimizar o custo de uma busca
- utiliza o custo até o objetivo h(n) e o custo de cada nó g(n)
- custo total a ser minimizado -> f(n) = g(n) + h(n)

### Modelando problema de roteamento

<img src="images/srs_campinas.png" width="800" align="center">

In [1]:
class Graph:

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()


    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist

    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance

    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


class Node:
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost    

    def __eq__(self, other):
        return self.name == other.name
    
    def __lt__(self, other):
         return self.f < other.f
    
    def __repr__(self):
        return ('({0},{1})'.format(self.name, self.f))

    def __hash__(self):
        return hash(self.name)

#### Populando grafo

In [2]:
graph = Graph()

graph.connect('SRS', 'Pouso_Alegre', 28.9)
graph.connect('Pouso_Alegre', 'Cambui', 49.1)
graph.connect('Pouso_Alegre', 'Congonhal', 24.3)
graph.connect('Pouso_Alegre', 'Borda', 28.8)
graph.connect('Cambui', 'Camanducaia', 24.7)
graph.connect('Camanducaia', 'Braganca', 60.4)
graph.connect('Braganca', 'Atibaia', 25.2)
graph.connect('Braganca', 'Itapira', 82.4)
graph.connect('Atibaia', 'Campinas', 65.6)
graph.connect('Itapira', 'Campinas', 70.7)
graph.connect('Borda', 'Jacutinga', 57.6)
graph.connect('Jacutinga', 'Itapira', 33.2)
graph.connect('Congonhal', 'Ipuiuna', 24.6)
graph.connect('Ipuiuna', 'Andradas', 67.5)
graph.connect('Andradas', 'ESPinhal', 28.4)
graph.connect('ESPinhal', 'MogiGuacu', 35.7)
graph.connect('MogiGuacu', 'MogiMirim', 25)
graph.connect('MogiMirim', 'Campinas', 60.1)

graph.make_undirected()

heuristics = {}
heuristics['SRS'] = 165
heuristics['Pouso_Alegre'] = 137
heuristics['Cambui'] = 108
heuristics['Camanducaia'] = 97
heuristics['Braganca'] = 54
heuristics['Atibaia'] = 57
heuristics['Campinas'] = 0
heuristics['Borda'] = 117
heuristics['Jacutinga'] = 84
heuristics['Itapira'] = 58
heuristics['Congonhal'] = 135
heuristics['Ipuiuna'] = 139
heuristics['Andradas'] = 106
heuristics['ESPinhal'] = 86
heuristics['MogiGuacu'] = 62
heuristics['MogiMirim'] = 54

### Implementando busca

In [9]:
import heapq
import numpy as np

def get_path(atual, inicial):
    '''
    Função para gerar o caminho percorrido entre dois estados
    Args:
        - atual: estado atual
        - inicial: estado inicial
    Return:
        - Caminho percorrido entre o estado inicial e final
    '''
    path = []
    while atual != inicial:
        path.append(atual.name + ': ' + str(atual.g))
        atual = atual.parent
    path.append(inicial.name + ': ' + str(inicial.g))

    return path[::-1]
        

def a_star(grafo, heuristicas, inicio, fim):
    '''
    Algoritmo a*, dado um grafo, um cojunto de heurísticas, um estado/nó inicial e um estado/nó final, 
    busca o caminho ótimo e completo
    Args:
        - grafo: grafo contendo os estados da busca
        - heuristicas: heuristica de cada estado
        - inicio: estado inicial
        - fim: estado final
    Return:
        - Melhor caminho entre estados inicial e final, se a solução foi encontrada, quantidade de estados visitados
    '''
    
    # Criando nó inicial e nó objetivo (não possuem pai)
    start_node = Node(inicio, None)
    goal_node = Node(fim, None)
    
    # Inicializando atributos g(0) = 0,  f(0) = 0 + h(0)
    start_node.g = 0
    start_node.h = heuristicas[inicio]
    start_node.f = start_node.g + start_node.h

    state_id = 0

    # Fila de prioridades
    fila = []
    heapq.heappush(fila, (start_node.f, state_id, start_node))

    # Nós criados
    created_nodes = []
    created_nodes.append(start_node)

    # Nós visitados
    visited_nodes = []
    
    # Controle
    solucao_encontrada = False
    path = None

    while len(fila) != 0:
        print("="*50)
        atual = heapq.heappop(fila)[2]
        visited_nodes.append(atual)
        print(f"Visitando {atual.name} - H: {atual.h} - G: {atual.g} - F: {atual.f}")

        if atual == goal_node:
            path = get_path(atual, start_node)
            solucao_encontrada = True
            break

        # Percorrendo todos os vizinhos do nó atual
        for nome_viz, dist_viz in grafo.get(atual.name).items():

            # Criando nó com o nome do vizinho e passando o estado atual como pai
            vizinho = Node(nome_viz, atual)

            # Verificar se a distância pai -> vizinho (g) é menor que 
            # um possível valor (g) do vizinho (caso já tenha sido visitado)

            # caso não haja valor de g para o estado atual a função retorna infinito
            g_pai_vizinho = atual.g + dist_viz
            
            g_vizinho = vizinho.g if vizinho in created_nodes else np.inf
                
            # Conectando pai ao novo nó
            if g_pai_vizinho < g_vizinho:
                
                vizinho.g = g_pai_vizinho
                vizinho.h = heuristicas[vizinho.name]
                vizinho.f = vizinho.g + vizinho.h
                
                created_nodes.append(vizinho)
                state_id += 1
                heapq.heappush(fila, (vizinho.f, state_id, vizinho))
    
        print("Estados disponíveis:")
        print(fila)
        
    return path, solucao_encontrada, visited_nodes 

### Main

In [14]:
path, solucao, nos_visitados = a_star(grafo = graph, heuristicas = heuristics, inicio = "SRS", fim = "Campinas")

Visitando SRS - H: 165 - G: 0 - F: 165
Estados disponíveis:
[(165.9, 1, (Pouso_Alegre,165.9))]
Visitando Pouso_Alegre - H: 137 - G: 28.9 - F: 165.9
Estados disponíveis:
[(174.7, 4, (Borda,174.7)), (188.2, 3, (Congonhal,188.2)), (186.0, 2, (Cambui,186.0))]
Visitando Borda - H: 117 - G: 57.7 - F: 174.7
Estados disponíveis:
[(186.0, 2, (Cambui,186.0)), (188.2, 3, (Congonhal,188.2)), (199.3, 5, (Jacutinga,199.3))]
Visitando Cambui - H: 108 - G: 78.0 - F: 186.0
Estados disponíveis:
[(188.2, 3, (Congonhal,188.2)), (199.3, 5, (Jacutinga,199.3)), (199.7, 6, (Camanducaia,199.7))]
Visitando Congonhal - H: 135 - G: 53.2 - F: 188.2
Estados disponíveis:
[(199.3, 5, (Jacutinga,199.3)), (199.7, 6, (Camanducaia,199.7)), (216.8, 7, (Ipuiuna,216.8))]
Visitando Jacutinga - H: 84 - G: 115.30000000000001 - F: 199.3
Estados disponíveis:
[(199.7, 6, (Camanducaia,199.7)), (216.8, 7, (Ipuiuna,216.8)), (206.5, 8, (Itapira,206.5))]
Visitando Camanducaia - H: 97 - G: 102.7 - F: 199.7
Estados disponíveis:
[(206.5,

In [15]:
print(path)

['SRS: 0', 'Pouso_Alegre: 28.9', 'Borda: 57.7', 'Jacutinga: 115.30000000000001', 'Itapira: 148.5', 'Campinas: 219.2']
